# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import config
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect(f"host=127.0.0.1 dbname=sparkifydb user={config.user}")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data/')

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR7G5I41187FB4CE6C,NaN,"London, England",NaN,Adam Ant,233.40363,1,SONHOTT12A8C13493C,Something Girls,1982


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id', 'title','artist_id', 'year', 'duration']].values[0].tolist()
song_data

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[['artist_id', 'artist_name', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data

['AR7G5I41187FB4CE6C', 'Adam Ant', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [28]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data/')

In [12]:
filepath_log = log_files[0]

In [13]:
df = pd.read_json(filepath_log, lines=True)
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Theodore,M,0,Smith,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540306e+12,154,None,200,1541290555796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,52
1,Professor Longhair,Logged In,Ann,F,0,Banks,214.20363,free,"Salt Lake City, UT",PUT,NextSong,1.540896e+12,124,Mean Ol'World,200,1541292603796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,99


In [14]:
df.shape

(189, 18)

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [15]:
# filtering for next song action because other actions are actions on the site (login, homepage)
df = df.loc[df['page'] == 'NextSong'].reset_index()
df.head(2)

,index,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,1,Professor Longhair,Logged In,Ann,F,0,Banks,214.20363,free,"Salt Lake City, UT",PUT,NextSong,1.540896e+12,124,Mean Ol'World,200,1541292603796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,99
1,3,Gary Hobbs,Logged In,Jahiem,M,1,Miles,245.52444,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,En Mi Mundo,200,1541300092796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43


In [16]:
df.shape

(144, 19)

In [17]:
t = pd.to_datetime(df['ts'], unit='ms')
df['ts'] = t
t.head()

0   2018-11-04 00:50:03.796
1   2018-11-04 02:54:52.796
2   2018-11-04 02:58:57.796
3   2018-11-04 03:02:20.796
4   2018-11-04 04:35:52.796
Name: ts, dtype: datetime64[ns]

In [18]:
df['ts'] = t
timestamp = df['ts'].apply(lambda x: x.timestamp()).values
hour = df['ts'].apply(lambda x: x.hour).values
day = df['ts'].apply(lambda x: x.day).values
weekofyear = df['ts'].apply(lambda x: x.weekofyear).values
month = df['ts'].apply(lambda x: x.month).values
year = df['ts'].apply(lambda x: x.year).values
weekday = df['ts'].apply(lambda x: x.weekday()).values

In [29]:
# list of timestamp
# time_data = [t[1].timestamp(), t[1].hour, t[1].day, t[1].weekofyear, t[1].month, t[1].year, t[1].weekday()]
column_data = [timestamp, hour, day, weekofyear, month, year, weekday]
column_labels = ['timestamp', 'hour', 'day', 'week_of_year', 'month', 'year', 'weekday']
time_dict = dict(zip(column_labels, column_data))

In [20]:
time_df = pd.DataFrame(time_dict)
# time_df = pd.DataFrame(data=[timestamp, hour, day, weekofyear, month, year, weekday], columns=[column_labels])
time_df.head()

,timestamp,hour,day,week_of_year,month,year,weekday
0,1.541293e+09,0,4,44,11,2018,6
1,1.541300e+09,2,4,44,11,2018,6
2,1.541300e+09,2,4,44,11,2018,6
3,1.541301e+09,3,4,44,11,2018,6
4,1.541306e+09,4,4,44,11,2018,6


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [21]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [22]:
df.head(2)

,index,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,1,Professor Longhair,Logged In,Ann,F,0,Banks,214.20363,free,"Salt Lake City, UT",PUT,NextSong,1.540896e+12,124,Mean Ol'World,200,2018-11-04 00:50:03.796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,99
1,3,Gary Hobbs,Logged In,Jahiem,M,1,Miles,245.52444,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,En Mi Mundo,200,2018-11-04 02:54:52.796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43


In [23]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']].drop_duplicates()

In [24]:
user_df.shape

(20, 5)

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [25]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [26]:
df.head(1)


,index,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,1,Professor Longhair,Logged In,Ann,F,0,Banks,214.20363,free,"Salt Lake City, UT",PUT,NextSong,1.540896e+12,124,Mean Ol'World,200,2018-11-04 00:50:03.796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,99


In [27]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    results = cur.execute(song_select, (row.song, row.artist, row.length))
    songid, artistid = results if results else None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [30]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.